In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install autogluon

import pandas as pd
import numpy as np
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from autogluon.timeseries.utils.forecast import get_forecast_horizon_index_ts_dataframe


In [4]:
# 데이터 가져오기

train_file_path = '/content/drive/MyDrive/open-2/train.csv'
test_file_path = '/content/drive/MyDrive/open-2/test.csv'
sample_file_path = '/content/drive/MyDrive/open-2/sample_submission.csv'


df = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)
sample_data = pd.read_csv(sample_file_path)

print(df)

                    ID   timestamp item corporation location  supply(kg)  \
0      TG_A_J_20190101  2019-01-01   TG           A        J         0.0   
1      TG_A_J_20190102  2019-01-02   TG           A        J         0.0   
2      TG_A_J_20190103  2019-01-03   TG           A        J     60601.0   
3      TG_A_J_20190104  2019-01-04   TG           A        J     25000.0   
4      TG_A_J_20190105  2019-01-05   TG           A        J     32352.0   
...                ...         ...  ...         ...      ...         ...   
59392  RD_F_J_20230227  2023-02-27   RD           F        J    452440.0   
59393  RD_F_J_20230228  2023-02-28   RD           F        J    421980.0   
59394  RD_F_J_20230301  2023-03-01   RD           F        J    382980.0   
59395  RD_F_J_20230302  2023-03-02   RD           F        J    477220.0   
59396  RD_F_J_20230303  2023-03-03   RD           F        J    427520.0   

       price(원/kg)  
0              0.0  
1              0.0  
2           1728.0  
3  

In [5]:
# item원핫인코딩
df = pd.get_dummies(df, columns=['item'], prefix='', prefix_sep='')

df.head()

,ID,timestamp,corporation,location,supply(kg),price(원/kg),BC,CB,CR,RD,TG
0,TG_A_J_20190101,2019-01-01,A,J,0.0,0.0,0,0,0,0,1
1,TG_A_J_20190102,2019-01-02,A,J,0.0,0.0,0,0,0,0,1
2,TG_A_J_20190103,2019-01-03,A,J,60601.0,1728.0,0,0,0,0,1
3,TG_A_J_20190104,2019-01-04,A,J,25000.0,1408.0,0,0,0,0,1
4,TG_A_J_20190105,2019-01-05,A,J,32352.0,1250.0,0,0,0,0,1


In [6]:
# 목표시계열 로그값 추가 근디 효과없어보임
df["log_target"] = np.log(df["price(원/kg)"])


/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [7]:
df['item_id'] = df.ID.str[0:6]

train_data = TimeSeriesDataFrame(df.drop(columns=['ID'])) # 멀티 인덱스
train_data = pd.get_dummies(train_data, columns=['corporation', 'location'])

print(train_data)

                    supply(kg)  price(원/kg)  BC  CB  CR  RD  TG  log_target  \
item_id timestamp                                                             
TG_A_J  2019-01-01         0.0          0.0   0   0   0   0   1        -inf   
        2019-01-02         0.0          0.0   0   0   0   0   1        -inf   
        2019-01-03     60601.0       1728.0   0   0   0   0   1    7.454720   
        2019-01-04     25000.0       1408.0   0   0   0   0   1    7.249926   
        2019-01-05     32352.0       1250.0   0   0   0   0   1    7.130899   
...                        ...          ...  ..  ..  ..  ..  ..         ...   
RD_F_J  2023-02-27    452440.0        468.0   0   0   0   1   0    6.148468   
        2023-02-28    421980.0        531.0   0   0   0   1   0    6.274762   
        2023-03-01    382980.0        574.0   0   0   0   1   0    6.352629   
        2023-03-02    477220.0        523.0   0   0   0   1   0    6.259581   
        2023-03-03    427520.0        529.0   0   0 

In [8]:
predictor = TimeSeriesPredictor(
    prediction_length=28,
    target="price(원/kg)",
    eval_metric="RMSE",
)

predictor.fit( train_data,
              random_seed=42,
              enable_ensemble = True,
)


================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'default',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price(원/kg)',
 'time_limit': None,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
INFO:lightning_fabric.utilities.seed:Global seed set to 42
AutoGluon will save models to AutogluonModels/ag-20231114_134304/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'price(원/kg)'
	past covariates:  ['supply(kg)', 

In [9]:
ld_board = predictor.leaderboard(train_data, silent=True)

Additional data provided, testing on additional data. Resulting leaderboard will be sorted according to test score (`score_test`).


In [10]:
ld_board

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time_marginal,fit_order
0,WeightedEnsemble,-727.746442,-727.927425,35.941498,75.015054,2.860167,7
1,DeepAR,-766.002098,-762.860641,0.524294,0.439590,128.612144,6
2,AutoETS,-797.059173,-797.059173,35.763853,50.588202,0.068120,4
3,RecursiveTabular,-810.212523,-810.212523,0.691430,1.934499,8.274991,5
4,Theta,-838.834688,-838.834688,34.480124,72.416087,0.072523,3
5,SeasonalNaive,-843.925825,-843.925825,0.117477,0.114397,0.062435,2
6,Naive,-997.063356,-997.063356,0.107863,0.110480,0.079641,1


In [11]:
# Retrain model on all of the data (training + validation).
predictor.refit_full()

Refitting models via `refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix '_FULL' and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `fit` call.
Fitting model: Naive_FULL | Skipping fit via cloning parent ...
Fitting model: SeasonalNaive_FULL | Skipping fit via cloning parent ...
Fitting model: Theta_FULL | Skipping fit via cloning parent ...
Fitting model: AutoETS_FULL | Skipping fit via cloning parent ...
Fitting model: RecursiveTabular_FULL
	6.33    s     = Training runtime
Fitting model: DeepAR_FULL | Skipping fit via cloning parent ...
Fitting model: WeightedEnsemble_FULL | Skipping fit via cloning parent ...
Refit complete. Models trained: ['Naive_FULL', 'SeasonalNaive_FULL', 'Theta_FULL', 'AutoETS_FULL', 'RecursiveTabular_FULL', 'DeepAR_FULL', 'WeightedEnsemble_FULL']
Total runtime: 6.40 s
Updated best model to 'WeightedEnsemble_FULL' (Previously 'W

{'Naive': 'Naive_FULL',
 'SeasonalNaive': 'SeasonalNaive_FULL',
 'Theta': 'Theta_FULL',
 'AutoETS': 'AutoETS_FULL',
 'RecursiveTabular': 'RecursiveTabular_FULL',
 'DeepAR': 'DeepAR_FULL',
 'WeightedEnsemble': 'WeightedEnsemble_FULL'}

In [12]:
# Return quantile and mean forecasts for the given dataset, starting from the end of each time series.

pred = predictor.predict(train_data, random_seed=42,)

INFO:lightning_fabric.utilities.seed:Global seed set to 42
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble_FULL


In [13]:
pred

mean          0.1          0.2          0.3  \
item_id timestamp                                                        
TG_A_J  2023-03-04  3303.832109  1384.758947  2050.959837  2507.506607   
        2023-03-05   619.871556 -1597.886000  -839.139141  -293.125073   
        2023-03-06  3094.916454   -21.172905  1078.291544  1842.413611   
        2023-03-07  3384.701295  -137.950683  1084.885260  1989.288555   
        2023-03-08  3338.841519  -478.494480   884.041286  1825.116838   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   520.106794   -97.721473   118.807781   269.034640   
        2023-03-28   519.110925  -130.615805   119.328849   270.738148   
        2023-03-29   517.419009  -153.914232   108.326279   267.626984   
        2023-03-30   494.706201  -184.218917    78.876139   248.721486   
        2023-03-31   505.181234  -177.685274    72.505638   243.658276   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
TG_A_J  2023-03-04  2918.596021  3279.469834  3664.104724  4070.224928   
        2023-03-05   169.032852   597.459944  1037.007131  1502.321923   
        2023-03-06  2495.092999  3091.011881  3716.359641  4397.242181   
        2023-03-07  2712.376243  3393.535360  4072.407853  4816.266612   
        2023-03-08  2613.717697  3343.778117  4067.281053  4850.939466   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   395.451514   515.183500   630.851948   756.331004   
        2023-03-28   406.832980   526.514541   652.053982   785.023815   
        2023-03-29   400.509954   522.102324   648.333277   783.475600   
        2023-03-30   384.148600   502.776796   632.596906   767.818251   
        2023-03-31   380.701239   513.064089   639.568407   781.928386   

                            0.8          0.9  
item_id timestamp                             
TG_A_J  2023-03-04  4545.204251  5237.470394  
        2023-03-05  2045.527880  2795.489713  
        2023-03-06  5147.741514  6201.062152  
        2023-03-07  5655.453886  6974.492565  
        2023-03-08  5786.022741  7209.278853  
...                         ...          ...  
RD_F_J  2023-03-27   923.660684  1151.757764  
        2023-03-28   935.453235  1159.257039  
        2023-03-29   949.273818  1167.490606  
        2023-03-30   929.046291  1152.913235  
        2023-03-31   941.739614  1166.397119  

[1092 rows x 10 columns]

In [14]:
sample_data['answer'] = pred.reset_index()['mean']
sample_data.loc[sample_data['answer'] < 0.0, 'answer'] = 0.0


In [15]:
sample_data

,ID,answer
0,TG_A_J_20230304,3303.832109
1,TG_A_J_20230305,619.871556
2,TG_A_J_20230306,3094.916454
3,TG_A_J_20230307,3384.701295
4,TG_A_J_20230308,3338.841519
...,...,...
1087,RD_F_J_20230327,520.106794
1088,RD_F_J_20230328,519.110925
1089,RD_F_J_20230329,517.419009
1090,RD_F_J_20230330,494.706201


In [16]:

# 'ID' 열에서 날짜 정보 추출
sample_data['date'] = pd.to_datetime(sample_data['ID'].str[-8:])

# 일요일 여부 확인 (dayofweek에서 6은 일요일)
sample_data['is_sunday'] = sample_data['date'].dt.dayofweek == 6

# 일요일에 해당하는 행의 'answer' 값을 0.0으로 설정
sample_data.loc[sample_data['is_sunday'], 'answer'] = 0.0

# 'date'와 'is_sunday' 열 삭제 (더 이상 필요 없음)
sample_data.drop(columns=['date', 'is_sunday'], inplace=True)

# 결과 확인
print(sample_data.head())

                ID       answer
0  TG_A_J_20230304  3303.832109
1  TG_A_J_20230305     0.000000
2  TG_A_J_20230306  3094.916454
3  TG_A_J_20230307  3384.701295
4  TG_A_J_20230308  3338.841519


In [17]:

save_path = '/content/drive/My Drive/angautosubmission2.csv'
sample_data.to_csv(save_path, index=False)

In [18]:
print(predictor.evaluate(train_data))
print(predictor.fit_summary())
print(predictor.get_model_best())
print(predictor.info())

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble_FULL


-739.3062714985451
****************** Summary of fit() ******************
Estimated performance of each model:
                    model   score_val  pred_time_val  fit_time_marginal  \
0        WeightedEnsemble -727.927425      75.015054           2.860167   
1                  DeepAR -762.860641       0.439590         128.612144   
2                 AutoETS -797.059173      50.588202           0.068120   
3        RecursiveTabular -810.212523       1.934499           8.274991   
4                   Theta -838.834688      72.416087           0.072523   
5           SeasonalNaive -843.925825       0.114397           0.062435   
6                   Naive -997.063356       0.110480           0.079641   
7   WeightedEnsemble_FULL         NaN            NaN           2.860167   
8              Theta_FULL         NaN            NaN           0.000215   
9      SeasonalNaive_FULL         NaN            NaN           0.000267   
10  RecursiveTabular_FULL         NaN            NaN           6